# 2.8	Praxisbeispiel Teil 2: Dateien herunterladen und extrahieren

Im letzten Schritt hast du gesehen, wie wir die URLs des Heise Newstickers extrahieren können. Nun werden die die einzelnen Seiten herunterladen und extrahieren.

## Seiten herunterladen

Um die Websiten zu akquirieren, lädst du im ersten Schritt die URLs ein, die du zuvor generiert hast:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f urls-2020.txt || wget https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/urls-2020.txt")

In [ ]:
urls = open("urls-2020.txt").read().split("\n")

Am besten speicherst du alle heruntergeladenen Artikel in einem eigenen Verzeichnis. Damit du das Notebook öfter laufen lassen kannst, legest du das Verzeichnis nur an, wenn es nicht bereits existiert:

In [ ]:
import os
if not os.path.isdir("artikel"):
    os.mkdir("artikel")

Bei größeren Downloads solltest du außerdem versuchen, nur Dateien herunterzuladen, die du nicht vorher schon heruntergeladen hattest. Dazu legst du am besten eine Funktion an, die aus einer URL einen Dateinamen konstruiert. Bei den Heise-Artikeln kannst du einfach den Teil der URL nach dem letzten `/` nehmen, grundsätzlich wären aber auch MD5-Hashes der URLs möglich:

In [ ]:
def article_filename(url):
    c = url.split("/")
    return "/".join(["artikel", c[-1]])

In dieser Schleife lädst du nun alle Seiten herunter, die noch nicht in dem Artikel-Verzeichnis existieren. Um von HTTP-Keepalive profitieren zu können, nutzt du die `Session` von `requests`. Trotzdem kann der Download eine ganze Weile dauern. Damit du immer siehst, wie der Fortschritt ist, kapselst du den Iterator in `tqdm`, einer sehr schönen Forschrittsanzeige:

In [ ]:
!pip install tqdm

In [ ]:
import requests
from tqdm.auto import tqdm

s = requests.Session()
for u in tqdm(urls):
    if u == '':
        continue
    filename = article_filename(u)
    if not os.path.isfile(filename):
        print(filename)
        print(u)
        r = s.get("https://www.heise.de" + u)
        open(filename, 'wb').write(r.content)    

`tqdm` nutzt du auch am besten für die Extraktion. Dazu kannst du einfach den Code aus *Lektion 6* übernehmen:

In [ ]:
import json
from bs4 import BeautifulSoup
res = []
for u in tqdm(urls):
    filename = article_filename(u)
    html = open(filename).read()
    soup = BeautifulSoup(html)
    d = {}
    d["title"] = soup.h1.text.strip()
    d["header"] = soup.select_one("#meldung > div.article-layout__header-container > header > p").text.strip()
    d["author"] = soup.select_one("a.redakteurskuerzel__link").attrs["title"]
    d["text"] = "\n".join([p.text.strip() 
                            for p in soup.select("#meldung > div.article-layout__content-container > div > p")])
    d["keywords"] = soup.find("meta", {"name": "keywords"})["content"]
    ld = json.loads(soup.find("script", type="application/ld+json").string)
    for k in ["identifier", "url", "datePublished", "commentCount"]:
        d[k] = ld[0][k]
    
    res.append(d)

Was mit einer Seite funktioniert, muss leider noch lange nicht für alle Seiten funktionieren...

Offenbar haben nicht alle Seiten einen Autor, auch ein Header ist nicht auf allen Seiten vorhanden. Wenn du weiter probierst, wirst du sehen, dass auch `ld+json`-Informationen nicht überall zu finden sind.

Um das richtig abzufangen, brauchst du ein Exception-Handling. Ganz richtigerweise solltest du nur bestimmte Exceptions abfangen. Da es und hier nicht um sauberes Exception-Handling geht, sondern wir den Code übersichtlich halten wollten, fangen wir alle Exceptions ab:

In [ ]:
res = []
for u in tqdm(urls):
    try:
        filename = article_filename(u)
        html = open(filename).read()
        soup = BeautifulSoup(html)
    except:
        # Datei nicht gefunden, invalides HTML etc.
        continue
    d = {}
    d["title"] = soup.h1.text.strip()
    try:
        d["header"] = soup.select_one("#meldung > div.article-layout__header-container > header > p").text.strip()
    except:
        pass
    try:
        d["author"] = soup.select_one("a.redakteurskuerzel__link").attrs["title"]
    except:
        pass
    d["text"] = "\n".join([p.text.strip() 
                            for p in soup.select("#meldung > div.article-layout__content-container > div > p")])
    try:
        d["keywords"] = soup.find("meta", {"name": "keywords"})["content"]
    except:
        pass
    try:
        ld = json.loads(soup.find("script", type="application/ld+json").string)
        for k in ["identifier", "url", "datePublished", "commentCount"]:
            d[k] = ld[0][k]
    except:
        pass
    
    res.append(d)

Nach ungefähr 10 Minuten müsste das bei dir durchgelaufen sein und du hast nun eine große Liste mit `dict`s aller Artikel zur Verfügung.

Diese wandelst du am besten in einen `DataFrame`:

In [ ]:
import pandas as pd
articles = pd.DataFrame(res)
len(articles)

Bei so vielen Artikeln solltest du unbedingt etwas Qualitätskontrolle betreiben. Welche Artikel haben keinen `identifier`?

In [ ]:
articles[articles["identifier"].isna()]

Gibt es welche ohne `url`?

In [ ]:
articles[articles["url"].isna()]

Das sind die gleichen, bei denen sind viele Felder nicht belegt und diese Dokumente kannst du ignorieren.

Deshalb kannst du den `DataFrame` so modifizieren, dass nur Dokumente übrig bleiben, deren `identifier` gesetzt ist. Dazu nutzt du die Methode `dropna`. 

In [ ]:
articles = articles.dropna(subset=["identifier"])
len(articles)

Wie erwartet ist also nur ein Artikel verschwunden (diese Anzahl kann sich bei dir ändern, *Heise räumt auf*).

Leider gibt es jetzt noch doppelte `identifier`, die kannst du mit `drop_duplicates` löschen. Anschließend kannst du den `index` des `DataFrame` auf den `identifier` setzen und in einen Integer wandeln:

In [ ]:
articles = articles.drop_duplicates(subset=["identifier"]).set_index("identifier")
articles.index = articles.index.astype(int)
len(articles)

Nun sind wieder ein paar Artikel verschwunden, es bleiben nur die mit einem eindeutigen `identifier` übrig.

Diese verbleibenden *sauberen Artikel* packst du jetzt in eine SQLite-Datenbank:

In [ ]:
import sqlite3
sql = sqlite3.connect("heise-articles-2020.db")
articles.to_sql("articles", sql, index_label="id", if_exists="replace")

Mit diesen Artikeln können wir in den folgenden Kapitel weiterarbeiten.